### On‑Time Delivery Rate (OTDR) per Supplier

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# ---------- Path helpers ----------
base = Path("cleaned_data") if Path("cleaned_data").exists() else Path("clean_data")
raw  = Path("raw_data")

def load_first(*paths, parse_dates=None):
    for p in paths:
        pth = Path(p)
        if pth.exists():
            return pd.read_csv(pth, parse_dates=parse_dates or [])
    raise FileNotFoundError(f"None of these files found: {paths}")

In [ ]:
# ---------- Load data ----------
deliveries = load_first(
    base / "delivery_logs_clean.csv",
    base / "delivery_logs.csv",
    raw  / "delivery_logs.csv",
    parse_dates=["order_date","expected_delivery_date","actual_delivery_date"]
)
suppliers = load_first(
    base / "suppliers_clean.csv",
    base / "suppliers.csv",
    raw  / "suppliers.csv"
)

In [ ]:
# Minimal columns check
req_deliv = {"supplier_id","delivery_status","expected_delivery_date"}
if not req_deliv.issubset(deliveries.columns):
    missing = req_deliv - set(deliveries.columns)
    raise KeyError(f"Missing required columns in deliveries: {missing}")

req_sup = {"supplier_id","supplier_name","country"}
if not req_sup.issubset(suppliers.columns):
    missing = req_sup - set(suppliers.columns)
    raise KeyError(f"Missing required columns in suppliers: {missing}")

In [ ]:
# ---------- Overall OTDR per supplier ----------
overall = (
    deliveries
    .groupby("supplier_id", as_index=False)
    .agg(
        total_deliveries=("delivery_status","size"),
        on_time_deliveries=("delivery_status", lambda s: (s == "On-Time").sum())
    )
)
overall["otdr_pct"] = (overall["on_time_deliveries"] / overall["total_deliveries"] * 100).round(2)

In [ ]:
# Add supplier name/country
overall = overall.merge(suppliers[["supplier_id","supplier_name","country"]], on="supplier_id", how="left")
overall = overall[["supplier_id","supplier_name","country","total_deliveries","on_time_deliveries","otdr_pct"]].sort_values("otdr_pct", ascending=False)

In [ ]:
# ---------- Monthly OTDR per supplier ----------
deliveries["month"] = deliveries["expected_delivery_date"].dt.to_period("M").astype(str)
monthly = (
    deliveries
    .groupby(["supplier_id","month"], as_index=False)
    .agg(
        total_deliveries=("delivery_status","size"),
        on_time_deliveries=("delivery_status", lambda s: (s == "On-Time").sum())
    )
)
monthly["otdr_pct_monthly"] = (monthly["on_time_deliveries"] / monthly["total_deliveries"] * 100).round(2)
monthly = monthly[["supplier_id","month","otdr_pct_monthly"]].sort_values(["supplier_id","month"])

In [ ]:
# ---------- Save KPI outputs ----------
base.mkdir(exist_ok=True, parents=True)
overall.to_csv(base / "kpi_otdr_overall.csv", index=False)
monthly.to_csv(base / "kpi_otdr_monthly.csv", index=False)

print("✅ Saved:")
print(f" - {base / 'kpi_otdr_overall.csv'}")
print(f" - {base / 'kpi_otdr_monthly.csv'}")

# Peek at top results
print("\nTop suppliers by OTDR:")
print(overall.head(10))


### Lead Time Variance & Fill Rate per Supplier (overall & monthly)

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

In [ ]:
base = Path("clean_data")
raw  = Path("raw_data")

def load_first(parse_dates=None, *paths):
    for p in paths:
        pth = Path(p)
        if pth.exists():
            return pd.read_csv(pth, parse_dates=parse_dates or [])
    raise FileNotFoundError(f"None of these files found: {paths}")

In [ ]:
# --- Load data ---
deliveries = load_first(
    ["order_date","expected_delivery_date","actual_delivery_date"],
    base / "delivery_logs_clean.csv",
    base / "delivery_logs.csv",
    raw  / "delivery_logs.csv",
)
otdr_overall = pd.read_csv(base / "kpi_otdr_overall.csv")
otdr_monthly = pd.read_csv(base / "kpi_otdr_monthly.csv")

In [ ]:
# --- Ensure needed columns ---
needed = {"supplier_id","order_date","expected_delivery_date","actual_delivery_date",
          "quantity_ordered","quantity_received","delivery_status"}
missing = needed - set(deliveries.columns)
if missing:
    raise KeyError(f"Missing columns in deliveries: {missing}")

In [ ]:
# --- Lead time (days): actual - order_date; exclude rows with missing actual ---
lt_df = deliveries.dropna(subset=["actual_delivery_date"]).copy()
lt_df["lead_time_days"] = (lt_df["actual_delivery_date"] - lt_df["order_date"]).dt.days

In [ ]:
# --- Overall KPIs per supplier ---
overall_lt = (
    lt_df.groupby("supplier_id", as_index=False)
         .agg(avg_lead_time_days=("lead_time_days","mean"),
              lead_time_std_days=("lead_time_days","std"),
              lead_time_p90_days=("lead_time_days", lambda s: np.percentile(s, 90)))
)

overall_fill = (
    deliveries.groupby("supplier_id", as_index=False)
              .agg(total_ordered=("quantity_ordered","sum"),
                   total_received=("quantity_received","sum"),
                   partial_deliveries=("delivery_status", lambda s: (s=="Partial").sum()),
                   total_deliveries=("delivery_status","size"))
)
overall_fill["fill_rate_pct"] = (overall_fill["total_received"] / overall_fill["total_ordered"] * 100).round(2)
overall_fill["partial_rate_pct"] = (overall_fill["partial_deliveries"] / overall_fill["total_deliveries"] * 100).round(2)

kpi_overall = (
    overall_fill.merge(overall_lt, on="supplier_id", how="left")
                .merge(otdr_overall[["supplier_id","otdr_pct"]], on="supplier_id", how="left")
)

In [ ]:
# --- Monthly KPIs per supplier ---
deliveries["month"] = deliveries["expected_delivery_date"].dt.to_period("M").astype(str)
lt_df["month"] = lt_df["actual_delivery_date"].dt.to_period("M").astype(str)

monthly_lt = (
    lt_df.groupby(["supplier_id","month"], as_index=False)
         .agg(avg_lead_time_days=("lead_time_days","mean"),
              lead_time_std_days=("lead_time_days","std"),
              lead_time_p90_days=("lead_time_days", lambda s: np.percentile(s, 90)))
)

monthly_fill = (
    deliveries.groupby(["supplier_id","month"], as_index=False)
              .agg(total_ordered=("quantity_ordered","sum"),
                   total_received=("quantity_received","sum"),
                   partial_deliveries=("delivery_status", lambda s: (s=="Partial").sum()),
                   total_deliveries=("delivery_status","size"))
)
monthly_fill["fill_rate_pct"] = (monthly_fill["total_received"] / monthly_fill["total_ordered"] * 100).round(2)
monthly_fill["partial_rate_pct"] = (monthly_fill["partial_deliveries"] / monthly_fill["total_deliveries"] * 100).round(2)

kpi_monthly = (
    monthly_fill.merge(monthly_lt, on=["supplier_id","month"], how="left")
                .merge(otdr_monthly, on=["supplier_id","month"], how="left")
                .rename(columns={"otdr_pct_monthly":"otdr_pct"})
                .sort_values(["supplier_id","month"])
)

In [ ]:
# --- Save individual KPI files ---
base.mkdir(exist_ok=True, parents=True)
kpi_overall.round(2).to_csv(base / "kpi_leadtime_fill_overall.csv", index=False)
kpi_monthly.round(2).to_csv(base / "kpi_leadtime_fill_monthly.csv", index=False)

In [ ]:
# --- Assemble Supplier Scorecard (Overall & Monthly) ---
score_overall = kpi_overall[[
    "supplier_id","total_deliveries","otdr_pct",
    "avg_lead_time_days","lead_time_std_days","lead_time_p90_days",
    "fill_rate_pct","partial_rate_pct","total_ordered","total_received"
]].round(2)

score_monthly = kpi_monthly[[
    "supplier_id","month","total_deliveries","otdr_pct",
    "avg_lead_time_days","lead_time_std_days","lead_time_p90_days",
    "fill_rate_pct","partial_rate_pct","total_ordered","total_received"
]].round(2)

score_overall.to_csv(base / "supplier_scorecard_overall.csv", index=False)
score_monthly.to_csv(base / "supplier_scorecard_monthly.csv", index=False)

print("Saved:")
print(f" - {base/'kpi_leadtime_fill_overall.csv'}")
print(f" - {base/'kpi_leadtime_fill_monthly.csv'}")
print(f" - {base/'supplier_scorecard_overall.csv'}")
print(f" - {base/'supplier_scorecard_monthly.csv'}")

print("\n Supplier Scorecard (overall) preview:")
print(score_overall.head(10))


### Supplier Risk Index (Overall & Monthly)

###  Supplier Risk Index (Overall & Monthly)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
# ---------- Paths ----------
base = Path("clean_data")
raw  = Path("raw_data")
base.mkdir(exist_ok=True, parents=True)

In [ ]:
# ---------- Load data ----------
score_overall = pd.read_csv(base / "supplier_scorecard_overall.csv")
score_monthly = pd.read_csv(base / "supplier_scorecard_monthly.csv")

In [ ]:
# Optional supplier metadata (name/country)
suppliers = None
for p in [base / "suppliers_clean.csv", base / "suppliers.csv", raw / "suppliers.csv"]:
    if p.exists():
        suppliers = pd.read_csv(p)
        break

In [ ]:


# ---------- Helpers ----------
WEIGHTS = {
    "otdr_pct": 0.40,           # higher is better
    "fill_rate_pct": 0.25,      # higher is better
    "lead_time_std_days": 0.20, # lower is better (invert after min-max)
    "partial_rate_pct": 0.15    # lower is better (invert after min-max)
}

def minmax_safe(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    s_min, s_max = s.min(), s.max()
    if not np.isfinite(s_min) or not np.isfinite(s_max):
        return pd.Series(0.5, index=s.index)  # neutral
    if np.isclose(s_max, s_min):
        return pd.Series(0.5, index=s.index)  # no spread -> neutral
    return (s - s_min) / (s_max - s_min)

def finalize_ranking(df: pd.DataFrame, group_cols=None) -> pd.DataFrame:
    out = df.copy()
    out["risk_index_score"] = pd.to_numeric(out["risk_index_score"], errors="coerce").fillna(0)

    # Dense rank (higher score = better rank #1)
    if group_cols:
        out["risk_rank"] = (
            out.groupby(group_cols)["risk_index_score"]
              .rank(method="dense", ascending=False)
              .fillna(0)
              .astype(int)
        )
        # Tiers per group (A best … E worst); if <5 unique, set N/A
        def tier_per_group(g):
            if g["risk_index_score"].nunique() >= 5:
                return pd.qcut(
                    g["risk_index_score"].rank(method="first", ascending=False),
                    q=5, labels=["A","B","C","D","E"]
                )
            return pd.Series(["N/A"]*len(g), index=g.index)
        out["risk_tier"] = out.groupby(group_cols, group_keys=False).apply(tier_per_group)
    else:
        out["risk_rank"] = (
            out["risk_index_score"].rank(method="dense", ascending=False)
                                   .fillna(0).astype(int)
        )
        if out["risk_index_score"].nunique() >= 5:
            out["risk_tier"] = pd.qcut(
                out["risk_index_score"].rank(method="first", ascending=False),
                q=5, labels=["A","B","C","D","E"]
            )
        else:
            out["risk_tier"] = "N/A"

    return out

def compute_index(df: pd.DataFrame, is_monthly: bool) -> pd.DataFrame:
    req_cols = ["supplier_id","otdr_pct","fill_rate_pct","lead_time_std_days","partial_rate_pct"]
    miss = [c for c in req_cols if c not in df.columns]
    if miss:
        raise KeyError(f"Missing required KPI columns: {miss}")

    # Ensure numeric
    df = df.copy()
    for c in ["otdr_pct","fill_rate_pct","lead_time_std_days","partial_rate_pct"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    if is_monthly:
        # Normalize within each month
        def per_month(g):
            g = g.copy()
            g["_otdr_n"]    = minmax_safe(g["otdr_pct"])
            g["_fill_n"]    = minmax_safe(g["fill_rate_pct"])
            g["_ltstd_n"]   = 1 - minmax_safe(g["lead_time_std_days"])  # invert (lower better)
            g["_partial_n"] = 1 - minmax_safe(g["partial_rate_pct"])    # invert (lower better)
            g["risk_index_score"] = (
                WEIGHTS["otdr_pct"]         * g["_otdr_n"] +
                WEIGHTS["fill_rate_pct"]    * g["_fill_n"] +
                WEIGHTS["lead_time_std_days"] * g["_ltstd_n"] +
                WEIGHTS["partial_rate_pct"] * g["_partial_n"]
            ) * 100
            return g
        scored = df.groupby("month", group_keys=False).apply(per_month)
        scored = finalize_ranking(scored, group_cols=["month"])
    else:
        # Normalize across all suppliers
        scored = df.copy()
        scored["_otdr_n"]    = minmax_safe(scored["otdr_pct"])
        scored["_fill_n"]    = minmax_safe(scored["fill_rate_pct"])
        scored["_ltstd_n"]   = 1 - minmax_safe(scored["lead_time_std_days"])  # invert
        scored["_partial_n"] = 1 - minmax_safe(scored["partial_rate_pct"])    # invert
        scored["risk_index_score"] = (
            WEIGHTS["otdr_pct"]         * scored["_otdr_n"] +
            WEIGHTS["fill_rate_pct"]    * scored["_fill_n"] +
            WEIGHTS["lead_time_std_days"] * scored["_ltstd_n"] +
            WEIGHTS["partial_rate_pct"] * scored["_partial_n"]
        ) * 100
        scored = finalize_ranking(scored, group_cols=None)

    # Attach names/country if available
    if suppliers is not None and {"supplier_id","supplier_name","country"}.issubset(suppliers.columns):
        scored = scored.merge(suppliers[["supplier_id","supplier_name","country"]],
                              on="supplier_id", how="left")

    # Select final columns
    keep = ["supplier_id"]
    if "supplier_name" in scored.columns: keep.append("supplier_name")
    if "country" in scored.columns: keep.append("country")
    if is_monthly: keep.append("month")
    keep += [
        "otdr_pct","fill_rate_pct","lead_time_std_days","partial_rate_pct",
        "risk_index_score","risk_rank","risk_tier"
    ]
    scored = scored[keep].sort_values(["risk_rank","supplier_id"]).round(2)
    return scored




In [ ]:
# ---------- Compute & Save ----------
risk_overall = compute_index(score_overall, is_monthly=False)
risk_monthly = compute_index(score_monthly, is_monthly=True)

risk_overall.to_csv(base / "supplier_risk_index_overall.csv", index=False)
risk_monthly.to_csv(base / "supplier_risk_index_monthly.csv", index=False)

print("✅ Saved:")
print(f" - {base/'supplier_risk_index_overall.csv'}")
print(f" - {base/'supplier_risk_index_monthly.csv'}")

print("\n🔎 Top 10 suppliers by Risk Index (Overall):")
print(risk_overall.head(10))

### Export Supplier Scorecard & Risk Index to Excel (with formatting)

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# Load Data
base_path = Path("clean_data")
delivery_logs = pd.read_csv(base_path / "delivery_logs.csv", parse_dates=["order_date", "expected_delivery_date", "actual_delivery_date"])
suppliers = pd.read_csv(base_path / "suppliers.csv")

In [ ]:
# Create month column from order_date
delivery_logs["month"] = pd.to_datetime(delivery_logs["order_date"]).dt.to_period("M").astype(str)

In [ ]:
# KPI 1: OTDR (On-Time Delivery Rate)
delivery_logs["on_time"] = delivery_logs["actual_delivery_date"] <= delivery_logs["expected_delivery_date"]

kpi_otdr_overall = (
    delivery_logs.groupby("supplier_id")
    .agg(total_deliveries=("delivery_id", "count"),
         on_time_deliveries=("on_time", "sum"))
    .reset_index()
)
kpi_otdr_overall["otdr_pct"] = (kpi_otdr_overall["on_time_deliveries"] / kpi_otdr_overall["total_deliveries"]) * 100

kpi_otdr_monthly = (
    delivery_logs.groupby(["supplier_id", "month"])
    .agg(total_deliveries=("delivery_id", "count"),
         on_time_deliveries=("on_time", "sum"))
    .reset_index()
)
kpi_otdr_monthly["otdr_pct"] = (kpi_otdr_monthly["on_time_deliveries"] / kpi_otdr_monthly["total_deliveries"]) * 100

kpi_otdr_overall.to_csv(base_path / "kpi_otdr_overall.csv", index=False)
kpi_otdr_monthly.to_csv(base_path / "kpi_otdr_monthly.csv", index=False)

In [ ]:
# KPI 2: Lead Time & Fill Rate
delivery_logs["lead_time_days"] = (delivery_logs["actual_delivery_date"] - delivery_logs["order_date"]).dt.days
delivery_logs["fill_rate"] = delivery_logs["quantity_received"] / delivery_logs["quantity_ordered"]
delivery_logs["partial_rate"] = (delivery_logs["quantity_received"] < delivery_logs["quantity_ordered"]).astype(int)

kpi_leadtime_fill_overall = (
    delivery_logs.groupby("supplier_id")
    .agg(avg_lead_time_days=("lead_time_days", "mean"),
         lead_time_std_days=("lead_time_days", "std"),
         lead_time_p90_days=("lead_time_days", lambda x: x.quantile(0.9)),
         fill_rate_pct=("fill_rate", lambda x: min(x.mean() * 100, 100)),  # cap at 100%
         partial_rate_pct=("partial_rate", lambda x: x.mean() * 100),
         total_ordered=("quantity_ordered", "sum"),
         total_received=("quantity_received", "sum"))
    .reset_index()
)

kpi_leadtime_fill_monthly = (
    delivery_logs.groupby(["supplier_id", "month"])
    .agg(avg_lead_time_days=("lead_time_days", "mean"),
         lead_time_std_days=("lead_time_days", "std"),
         lead_time_p90_days=("lead_time_days", lambda x: x.quantile(0.9)),
         fill_rate_pct=("fill_rate", lambda x: min(x.mean() * 100, 100)),
         partial_rate_pct=("partial_rate", lambda x: x.mean() * 100),
         total_ordered=("quantity_ordered", "sum"),
         total_received=("quantity_received", "sum"))
    .reset_index()
)

kpi_leadtime_fill_overall.to_csv(base_path / "kpi_leadtime_fill_overall.csv", index=False)
kpi_leadtime_fill_monthly.to_csv(base_path / "kpi_leadtime_fill_monthly.csv", index=False)

In [ ]:
# Merge KPIs for Supplier Scorecard
supplier_scorecard_overall = (
    kpi_otdr_overall.merge(kpi_leadtime_fill_overall, on="supplier_id", how="left")
    .merge(suppliers, on="supplier_id", how="left")
)

supplier_scorecard_monthly = (
    kpi_otdr_monthly.merge(kpi_leadtime_fill_monthly, on=["supplier_id", "month"], how="left")
    .merge(suppliers, on="supplier_id", how="left")
)

supplier_scorecard_overall.to_csv(base_path / "supplier_scorecard_overall.csv", index=False)
supplier_scorecard_monthly.to_csv(base_path / "supplier_scorecard_monthly.csv", index=False)

In [ ]:
# Supplier Risk Index
def compute_risk_index(row):
    score = (row["otdr_pct"] * 0.4) + (row["fill_rate_pct"] * 0.3) - (row["partial_rate_pct"] * 0.1) - (row["lead_time_std_days"] * 0.2)
    return score

supplier_scorecard_overall["risk_index_score"] = supplier_scorecard_overall.apply(compute_risk_index, axis=1)
supplier_scorecard_overall["risk_rank"] = supplier_scorecard_overall["risk_index_score"].rank(ascending=False).fillna(0).astype(int)

def assign_tier(rank):
    if rank <= 3:
        return "A"
    elif rank <= 6:
        return "B"
    else:
        return "C"

supplier_scorecard_overall["risk_tier"] = supplier_scorecard_overall["risk_rank"].apply(assign_tier)

supplier_scorecard_monthly["risk_index_score"] = supplier_scorecard_monthly.apply(compute_risk_index, axis=1)
supplier_scorecard_monthly["risk_rank"] = supplier_scorecard_monthly.groupby("month")["risk_index_score"].rank(ascending=False).fillna(0).astype(int)
supplier_scorecard_monthly["risk_tier"] = supplier_scorecard_monthly["risk_rank"].apply(assign_tier)

supplier_scorecard_overall.to_csv(base_path / "supplier_risk_index_overall.csv", index=False)
supplier_scorecard_monthly.to_csv(base_path / "supplier_risk_index_monthly.csv", index=False)

print(" Saved all KPI and risk index files with capped fill_rate_pct and fixed NaN ranks.")


### Root‑Cause Analysis for Low OTDR / Low Fill Rate

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

BASE = Path("clean_data")
RAW  = Path("raw_data")
BASE.mkdir(parents=True, exist_ok=True)

In [ ]:
# 1) Load datasets (robust)
score_m = pd.read_csv(BASE / "supplier_scorecard_monthly.csv")
dl = pd.read_csv(
    (BASE / "delivery_logs.csv") if (BASE / "delivery_logs.csv").exists() else (RAW / "delivery_logs.csv"),
    parse_dates=["order_date","expected_delivery_date","actual_delivery_date"]
)

suppliers = None
for p in [BASE/"suppliers.csv", BASE/"suppliers_clean.csv", RAW/"suppliers.csv"]:
    if p.exists():
        suppliers = pd.read_csv(p)
        break

components = None
for p in [BASE/"components.csv", BASE/"components_clean.csv", RAW/"components.csv"]:
    if p.exists():
        components = pd.read_csv(p)
        break

In [ ]:
# Sanity checks
need_score_cols = {"supplier_id","month","otdr_pct","fill_rate_pct","total_deliveries","total_ordered","total_received"}
miss = need_score_cols - set(score_m.columns)
if miss:
    raise KeyError(f"supplier_scorecard_monthly.csv missing columns: {miss}")

need_dl_cols = {"delivery_id","supplier_id","component_id","expected_delivery_date","actual_delivery_date",
                "quantity_ordered","quantity_received","delivery_status"}
miss = need_dl_cols - set(dl.columns)
if miss:
    raise KeyError(f"delivery_logs.csv missing columns: {miss}")

In [ ]:
# 2) Thresholds & failing months
OTDR_MIN = 85.0
FILL_MIN = 95.0

failing = score_m[
    (score_m["otdr_pct"] < OTDR_MIN) | (score_m["fill_rate_pct"] < FILL_MIN)
].copy()

# Keep tidy columns
keep_cols = ["supplier_id","month","total_deliveries","otdr_pct","fill_rate_pct",
             "total_ordered","total_received","avg_lead_time_days","lead_time_std_days","lead_time_p90_days"]
failing = failing[ [c for c in keep_cols if c in failing.columns] ].reset_index(drop=True)

In [ ]:
# 3) Join delivery detail for failing supplier-months
#     - derive month on expected date (supplier KPIs used expected date timeline)
dl["month"] = dl["expected_delivery_date"].dt.to_period("M").astype(str)
key = failing[["supplier_id","month"]].drop_duplicates()
dl_fail = dl.merge(key, on=["supplier_id","month"], how="inner").copy()

# Lateness (days); late if actual > expected. If no actual, treat as late with NaN days.
dl_fail["is_late"] = np.where(
    dl_fail["actual_delivery_date"].notna() & (dl_fail["actual_delivery_date"] > dl_fail["expected_delivery_date"]),
    1, 0
)
# Lateness days only for rows with actual date
dl_fail["late_days"] = np.where(
    dl_fail["actual_delivery_date"].notna(),
    (dl_fail["actual_delivery_date"] - dl_fail["expected_delivery_date"]).dt.days.clip(lower=0),
    np.nan
)

# Partial fill driver
dl_fail["is_partial"] = (dl_fail["delivery_status"] == "Partial").astype(int)

# Proxy "cause" buckets (if no reason codes exist)
def bucket_cause(row):
    if row["is_partial"] == 1:
        return "Partial fulfillment"
    if pd.isna(row["actual_delivery_date"]):
        return "Undelivered (open)"
    if row["late_days"] >= 14:
        return "Severe delay (14d+)"
    if 1 <= row["late_days"] < 14:
        return "Moderate delay (1–13d)"
    return "On-time"
dl_fail["delay_bucket"] = dl_fail.apply(bucket_cause, axis=1)

# Optional enrich: supplier name & country
if suppliers is not None and {"supplier_id","supplier_name","country"}.issubset(suppliers.columns):
    dl_fail = dl_fail.merge(suppliers[["supplier_id","supplier_name","country"]], on="supplier_id", how="left")

# Optional enrich: component category
if components is not None and {"component_id","category"}.issubset(components.columns):
    dl_fail = dl_fail.merge(components[["component_id","category"]], on="component_id", how="left")

In [ ]:
# 4) Summaries for visualization
# Supplier × Month summary
sup_month = (
    dl_fail.groupby(["supplier_id","month"], as_index=False)
           .agg(
                deliveries=("delivery_id","size"),
                late_deliveries=("is_late","sum"),
                partial_deliveries=("is_partial","sum"),
                total_ordered=("quantity_ordered","sum"),
                total_received=("quantity_received","sum"),
                late_days_avg=("late_days","mean"),
                late_days_p90=("late_days", lambda s: np.nanpercentile(s.dropna(), 90) if s.notna().any() else np.nan)
           )
)

In [ ]:
# Join KPI ratios back for context
sup_month = sup_month.merge(failing, on=["supplier_id","month"], how="left", suffixes=("","_kpi"))
sup_month["late_rate_pct"] = (sup_month["late_deliveries"] / sup_month["deliveries"] * 100).round(2)
sup_month["partial_rate_pct_det"] = (sup_month["partial_deliveries"] / sup_month["deliveries"] * 100).round(2)

In [ ]:
# Country × Month (if country available)
country_month = pd.DataFrame()
if "country" in dl_fail.columns:
    country_month = (
        dl_fail.groupby(["country","month"], as_index=False)
               .agg(
                    deliveries=("delivery_id","size"),
                    late_deliveries=("is_late","sum"),
                    partial_deliveries=("is_partial","sum"),
                    total_ordered=("quantity_ordered","sum"),
                    total_received=("quantity_received","sum"),
                    late_days_avg=("late_days","mean"),
                    late_days_p90=("late_days", lambda s: np.nanpercentile(s.dropna(), 90) if s.notna().any() else np.nan)
               )
    )
    country_month["late_rate_pct"] = (country_month["late_deliveries"]/country_month["deliveries"]*100).round(2)
    country_month["partial_rate_pct_det"] = (country_month["partial_deliveries"]/country_month["deliveries"]*100).round(2)

In [ ]:
# Category × Month (if category available)
category_month = pd.DataFrame()
if "category" in dl_fail.columns:
    category_month = (
        dl_fail.groupby(["category","month"], as_index=False)
               .agg(
                    deliveries=("delivery_id","size"),
                    late_deliveries=("is_late","sum"),
                    partial_deliveries=("is_partial","sum"),
                    total_ordered=("quantity_ordered","sum"),
                    total_received=("quantity_received","sum"),
                    late_days_avg=("late_days","mean"),
                    late_days_p90=("late_days", lambda s: np.nanpercentile(s.dropna(), 90) if s.notna().any() else np.nan)
               )
    )
    category_month["late_rate_pct"] = (category_month["late_deliveries"]/category_month["deliveries"]*100).round(2)
    category_month["partial_rate_pct_det"] = (category_month["partial_deliveries"]/category_month["deliveries"]*100).round(2)

In [ ]:
# 5) Save outputs
failing.to_csv(BASE / "rca_low_perf_monthly.csv", index=False)
dl_fail.to_csv(BASE / "rca_late_deliveries_detail.csv", index=False)
sup_month.round(2).to_csv(BASE / "rca_summary_by_supplier_month.csv", index=False)
if not country_month.empty:
    country_month.round(2).to_csv(BASE / "rca_summary_by_country_month.csv", index=False)
if not category_month.empty:
    category_month.round(2).to_csv(BASE / "rca_summary_by_category_month.csv", index=False)

# Preview key results
print(" Saved RCA files in clean_data/")
print(" - rca_low_perf_monthly.csv  (supplier-months failing thresholds)")
print(" - rca_late_deliveries_detail.csv  (row-level late drivers)")
print(" - rca_summary_by_supplier_month.csv  (roll-up for charts)")
if not country_month.empty:  print(" - rca_summary_by_country_month.csv")
if not category_month.empty: print(" - rca_summary_by_category_month.csv")

print("\n Top delay buckets driving failures:")
print(dl_fail["delay_bucket"].value_counts().head(10))
